# Innovation Degree of Freedom
   This file calculates the degree of freedom of innovation (defined as Num(immediate potential objects can be made) - Num(objs already made)) at different time points in the family history

In [1]:
%matplotlib inline

import os, re, glob, datetime, json
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

load empowerment dictionary

In [2]:
with open('tech_outputs/products.txt') as json_file:
    emp = json.load(json_file)

In [3]:
emp = {int(k): v for k, v in emp.items() if len(v)!= 0}

load map change data

In [4]:
data = pd.read_csv('../2_demographics/outputs/parsed_map_change.csv', index_col = [0])

In [5]:
data.head()

,time,locX,locY,obj,playerID,family
0,122.31,-5137,-1395,0,2276905,time-1573895755_eve-2276905_name-ZABICKI
1,125.06,-5138,-1394,0,2276905,time-1573895755_eve-2276905_name-ZABICKI
2,127.21,-5138,-1395,236,2276905,time-1573895755_eve-2276905_name-ZABICKI
3,129.09,-5135,-1396,134,2276905,time-1573895755_eve-2276905_name-ZABICKI
4,130.33,-5136,-1395,0,2276905,time-1573895755_eve-2276905_name-ZABICKI


In [6]:
len(data)

840408

find object and family index

In [7]:
data['cum_obj_num'] = data.groupby('family')['obj'].cumcount()

In [8]:
data['fam_idx'] = data.groupby('family',as_index = False).ngroup()

In [9]:
obj_list = data.groupby('family')['obj'].apply(list).to_list()

In [10]:
len(obj_list)

48

tag cumulative objects up to time t

In [ ]:
data['cum_objs'] = data.apply(lambda x: set(obj_list[x.fam_idx][0:x.cum_obj_num+1]), axis = 1)

In [24]:
data.head()

,time,locX,locY,obj,playerID,family,cum_obj_num,fam_idx,cum_objs
0,122.31,-5137,-1395,0,2276905,time-1573895755_eve-2276905_name-ZABICKI,0,0,{0}
1,125.06,-5138,-1394,0,2276905,time-1573895755_eve-2276905_name-ZABICKI,1,0,{0}
2,127.21,-5138,-1395,236,2276905,time-1573895755_eve-2276905_name-ZABICKI,2,0,"{0, 236}"
3,129.09,-5135,-1396,134,2276905,time-1573895755_eve-2276905_name-ZABICKI,3,0,"{0, 236, 134}"
4,130.33,-5136,-1395,0,2276905,time-1573895755_eve-2276905_name-ZABICKI,4,0,"{0, 236, 134}"


take only rows where new objects are made

In [28]:
data_unique = data.copy()

In [29]:
data_unique['shifted'] = data_unique['cum_objs'].shift()

In [32]:
data_unique.head()

,time,locX,locY,obj,playerID,family,cum_obj_num,fam_idx,cum_objs,shifted
0,122.31,-5137,-1395,0,2276905,time-1573895755_eve-2276905_name-ZABICKI,0,0,{0},NaN
1,125.06,-5138,-1394,0,2276905,time-1573895755_eve-2276905_name-ZABICKI,1,0,{0},{0}
2,127.21,-5138,-1395,236,2276905,time-1573895755_eve-2276905_name-ZABICKI,2,0,"{0, 236}",{0}
3,129.09,-5135,-1396,134,2276905,time-1573895755_eve-2276905_name-ZABICKI,3,0,"{0, 236, 134}","{0, 236}"
4,130.33,-5136,-1395,0,2276905,time-1573895755_eve-2276905_name-ZABICKI,4,0,"{0, 236, 134}","{0, 236, 134}"


In [34]:
data_unique['num_obj'] = data_unique['cum_objs'].apply(lambda x: len(x))

In [40]:
data_new = data_unique.groupby(['family', 'num_obj'])[['time', 'cum_objs']].apply(min).reset_index()

In [47]:
data_new.head()

,family,num_obj,time,cum_objs
0,time-1573895755_eve-2276905_name-ZABICKI,1,122.31,{0}
1,time-1573895755_eve-2276905_name-ZABICKI,2,127.21,"{0, 236}"
2,time-1573895755_eve-2276905_name-ZABICKI,3,129.09,"{0, 236, 134}"
3,time-1573895755_eve-2276905_name-ZABICKI,4,172.11,"{0, 2873, 236, 134}"
4,time-1573895755_eve-2276905_name-ZABICKI,5,175.11,"{0, 134, 236, 2742, 2873}"


In [44]:
len(data_new)

15277

find innovation degree of freedom

In [107]:
def find_emp(objs):
    tot = objs
    for i in objs:
        if i in emp.keys():
            tot = np.append(tot, [i for i in emp[i] if i not in tot])
    return len(tot) - len(objs)

In [108]:
data_new['degree_of_freedom'] = data_new['cum_objs'].apply(find_emp)

In [109]:
data_new.head()

,family,num_obj,time,cum_objs,emp,degree_of_freedom
0,time-1573895755_eve-2276905_name-ZABICKI,1,122.31,{0},0,0
1,time-1573895755_eve-2276905_name-ZABICKI,2,127.21,"{0, 236}",29,28
2,time-1573895755_eve-2276905_name-ZABICKI,3,129.09,"{0, 236, 134}",33,31
3,time-1573895755_eve-2276905_name-ZABICKI,4,172.11,"{0, 2873, 236, 134}",34,31
4,time-1573895755_eve-2276905_name-ZABICKI,5,175.11,"{0, 134, 236, 2742, 2873}",34,30


In [117]:
data_new.to_csv('empowerment_dynamic.csv')

In [112]:
test = data_new.groupby('family')['emp'].apply(max)

In [113]:
test

family
time-1573895755_eve-2276905_name-ZABICKI            1415
time-1573895904_eve-2276911_name-ROHRER              312
time-1573896101_eve-2276917_name-BADA               1431
time-1573896134_eve-2276919_name-SARINA             1627
time-1573896495_eve-2276927_name-LISSADE            2212
time-1573905001_eve-2277238_name-NORTH               466
time-1573906131_eve-2277285_name-NEW                 203
time-1573907431_eve-2277342_name-FINCH              2114
time-1573909483_eve-2277421_name-ISPAS              2024
time-1573913113_eve-2277579_name-IN                  262
time-1573914221_eve-2277632_name-WILD                914
time-1573916555_eve-2277781_name-AA                    3
time-1573918105_eve-2277860_name-RUTH                828
time-1573922544_eve-2278118_name-FUGLSANG              8
time-1573922618_eve-2278121_name-ZAZA                150
time-1573923283_eve-2278172_name-POPS                258
time-1573923963_eve-2278242_name-DESTINO             204
time-1573928065_eve-2278